In [187]:
import pandas as pd 

from sklearn.datasets import load_iris as load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


from matplotlib.pyplot import plot, show

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

In [188]:
class feature_eng(dict):
    def __init__(self, data):
        self.data = data
        
    def show_columns_with_null(self):
        lst = []
        for i in data.columns:
            if data[i].isna().sum():
                lst.append(i)
                print(f'{i} : {self.data[i].isna().sum()} null')
        print(f'tatol columns with null {len(lst)}')
        return lst
    
    def show_columns_are_ojebct(self):
        lst=[]
        for i in data.columns:
            if data[i].dtype == 'object':
                lst.append(i)
                print(f'{i} with {self.data[i].isna().sum()} nulls')
        print(f'tatol columns with null {len(lst)}')
        return lst
    
    def fill_null(self,field):
        if self.data[field].dtype == 'object':
            self.data[field].fillna('*',inplace=True)
        elif input(f"field type is {self.data[field].dtype}, do u want to fill with median?") == "yes":
            self.data[field].fillna(self.data[field].median(), inplace=True) 
            print(f'median of {field} column is {self.data[field].median()}')
        else:
            return self.data

            
    def make_one_hot(self, field):
        temp = pd.get_dummies(self.data[field], prefix=field)
        print(temp)
#       self.data.drop(field, axis=1, inplace=True, errors='ignore')
        return pd.concat([self.data, temp], axis=1)
    
    def labelencode(self,field):
        if self.data[field].dtype != 'object':
            print(f'type of this field is {self.data[field].dtype}')
            return self.data
        print(self.data[field].value_counts())
        print(f'number of titles : {len(self.data[field].value_counts())}')
        self.data[field] = LabelEncoder().fit_transform(data.Street) if input('do you want to proceed?') == 'yes' else self.data[field] 
        return self.data
    
    def labelencode_ordering(self,field):
        pass
    
    def impute(self):
        self.data = IterativeImputer().fit_transform(self.data)
        return self.data
        

In [189]:
train = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('./house-prices-advanced-regression-techniques/test.csv')

In [190]:
# train.describe()
# train.info()
# test.info()
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [191]:
x_train = train.drop('SalePrice', axis=1, errors='ignore')
data = pd.concat([x_train, test])
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [192]:
ins = feature_eng(data)
object_list = ins.show_columns_are_ojebct()
ins.data.head(10)

MSZoning with 4 nulls
Street with 0 nulls
Alley with 2721 nulls
LotShape with 0 nulls
LandContour with 0 nulls
Utilities with 2 nulls
LotConfig with 0 nulls
LandSlope with 0 nulls
Neighborhood with 0 nulls
Condition1 with 0 nulls
Condition2 with 0 nulls
BldgType with 0 nulls
HouseStyle with 0 nulls
RoofStyle with 0 nulls
RoofMatl with 0 nulls
Exterior1st with 1 nulls
Exterior2nd with 1 nulls
MasVnrType with 24 nulls
ExterQual with 0 nulls
ExterCond with 0 nulls
Foundation with 0 nulls
BsmtQual with 81 nulls
BsmtCond with 82 nulls
BsmtExposure with 82 nulls
BsmtFinType1 with 79 nulls
BsmtFinType2 with 80 nulls
Heating with 0 nulls
HeatingQC with 0 nulls
CentralAir with 0 nulls
Electrical with 1 nulls
KitchenQual with 1 nulls
Functional with 2 nulls
FireplaceQu with 1420 nulls
GarageType with 157 nulls
GarageFinish with 159 nulls
GarageQual with 159 nulls
GarageCond with 159 nulls
PavedDrive with 0 nulls
PoolQC with 2909 nulls
Fence with 2348 nulls
MiscFeature with 2814 nulls
SaleType wi

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal


In [193]:
for i in object_list:
    ins.fill_null(i)
ins.data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,*,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,*,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,*,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,*,IR1,Lvl,AllPub,...,0,0,*,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,1,2008,WD,Normal


In [195]:
ins.make_one_hot('MSZoning')
print(object_list[0])
ins.data.head(10)

      MSZoning_*  MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  \
0              0                 0            0            0            1   
1              0                 0            0            0            1   
2              0                 0            0            0            1   
3              0                 0            0            0            1   
4              0                 0            0            0            1   
...          ...               ...          ...          ...          ...   
1454           0                 0            0            0            0   
1455           0                 0            0            0            0   
1456           0                 0            0            0            1   
1457           0                 0            0            0            1   
1458           0                 0            0            0            1   

      MSZoning_RM  
0               0  
1               0  
2              

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,*,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,*,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,*,0,12,2008,WD,Normal
5,6,50,RL,85.0,14115,Pave,*,IR1,Lvl,AllPub,...,0,0,*,MnPrv,Shed,700,10,2009,WD,Normal
6,7,20,RL,75.0,10084,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,8,2007,WD,Normal
7,8,60,RL,NaN,10382,Pave,*,IR1,Lvl,AllPub,...,0,0,*,*,Shed,350,11,2009,WD,Normal
8,9,50,RM,51.0,6120,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,4,2008,WD,Abnorml
9,10,190,RL,50.0,7420,Pave,*,Reg,Lvl,AllPub,...,0,0,*,*,*,0,1,2008,WD,Normal
